In [219]:
import os
import warnings
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline


warnings.filterwarnings("ignore", category=UserWarning)

pd.set_option('display.max_rows', 50)  # показывать больше строк
pd.set_option('display.max_columns', 50)  # показывать больше колонок

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [220]:
def boxplot(col):
    fig, axes = plt.subplots(figsize = (14, 4))
    sns.boxplot(x='default', y=col, data=data[data['sample']==1],ax=axes)
    axes.set_title('Boxplot for ' + col)
    plt.show()

Загрузка данных и первичный осмотр

In [221]:
train = pd.read_csv('/kaggle/input/sf-dst-scoring/train.csv')
test = pd.read_csv('/kaggle/input/sf-dst-scoring/test.csv')
sample = pd.read_csv('/kaggle/input/sf-dst-scoring/sample_submission.csv')

In [222]:
print(train.info())
print()
print('Train size: ', train.shape)
print()
train.head()

In [223]:
print(sample.info())
print()
print('Sample_submission size: ', sample.shape)
print()
sample.head()

In [224]:
print(test.info())
print()
print('Test size: ', test.shape)
print()
test.head()

Описания полей датасета    
    
client_id - идентификатор клиента    
education - уровень образования    
sex - пол заемщика        
age - возраст заемщика     
car - флаг наличия автомобиля     
car_type - флаг автомобиля иномарки      
decline_app_cnt - количество отказанных прошлых заявок     
good_work - флаг наличия “хорошей” работы      
bki_request_cnt - количество запросов в БКИ     
home_address - категоризатор домашнего адреса     
work_address - категоризатор рабочего адреса     
income - доход заемщика     
foreign_passport - наличие загранпаспорта     
sna - связь заемщика с клиентами банка     
first_time - давность наличия информации о заемщике      
score_bki - скоринговый балл по данным из БКИ      
region_rating - рейтинг региона      
app_date - дата подачи заявки       
default - флаг дефолта по кредиту     

Посмотрим на целевую переменную default

In [225]:
sns.countplot(train['default'])

Выборка несбалансированная

Объеденим датасеты, чтобы удобнее было производить подготовку данных для модели

In [226]:
train['sample'] = 1   
test['sample'] = 0    
test['default'] = -1  
data = train.append(test, sort=False).reset_index(drop=True)

In [227]:
print(data.info())
print()
print('Data size: ', data.shape)
print()
data.head()

Пропуски присутствуют  в колонке education, посмотрим на распределение значений в ней

In [228]:
data.education.value_counts()

По старой доброй традиции заполним их популярным значением

In [229]:
data.education = data.education.fillna('SCH')

Распределим колонки по типу признака на числовые, бинарные и категориальные

In [230]:
num_cols = ['age','decline_app_cnt','score_bki','income','bki_request_cnt','region_rating'] 
cat_cols = ['education','work_address','home_address','sna','first_time'] 
bin_cols = ['sex','car','car_type','good_work','foreign_passport'] 

In [231]:
data[num_cols].hist(figsize=(25,10),bins=100)

Явно напрашивается прологорифмировать некоторые признаки, чтобы приблизить размпределение к нормальному

In [232]:
for i in ['age', 'bki_request_cnt', 'decline_app_cnt', 'income']:
    data[i] = np.log(data[i]+1)
data[num_cols].hist(figsize=(25,10),bins=100)

Улучшения есть, в разной степени, но есть

In [233]:
for col in num_cols:
    boxplot(col)

дефолт больше характерен для более молодых    
высокий score_bki характерен для дефолта    
рейтинг региона влияет на целевую переменную     
частые запросы в БКИ характерны для менее уверенных клиентов и могут сигнализировать дефолтную ситуацию    
в среднем, более высокий доход свидетельствует о меньшей вероятности дефолта    

In [234]:
sns.heatmap(data[num_cols].corr())

Корреляция незначительна

In [235]:
imp_num = pd.Series(f_classif(data[num_cols], data['default'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

Оценка значимости показывает наиболее значимым score_bki, давайте добавим новых признаков и оценим ещё раз

In [236]:
#поработаем с колонкой app_date
# получим кол-во дней от "начала эпохи" датасета
data['app_date'] = pd.to_datetime(data['app_date'], format='%d%b%Y')
data_min = min(data['app_date'])
data['days'] = (data['app_date'] - data_min).dt.days.astype('int')
data['day'] = data['app_date'].dt.day
data['month'] = data['app_date'].dt.month

data.drop(['app_date'],  axis = 1, inplace = True)

In [237]:
# средний доход по возрасту
mean_income = data.groupby('age')['income'].mean().to_dict()
data['mean_income_age'] = data['age'].map(mean_income)

# максимальный доход по возрасту
max_income = data.groupby('age')['income'].max().to_dict()
data['max_income_age'] = data['age'].map(max_income)

# нормализованный доход
data["normalized_income"] = abs((data.income - data.mean_income_age)/data.max_income_age)

# запросы по возрасту
mean_bki = data.groupby('age')['bki_request_cnt'].mean().to_dict()
data['mean_requests_age'] = data['age'].map(mean_bki)

# запросы по доходу
mean_bki_inc = data.groupby('income')['bki_request_cnt'].mean().to_dict()
data['mean_requests_income'] = data['income'].map(mean_bki_inc)

# доход по региону
mean_income_rat = data.groupby('region_rating')['income'].mean().to_dict()
data['mean_income_region'] = data['region_rating'].map(mean_income_rat)

# вытащим два столбца из датасета
data_addresses = data[['work_address', 'home_address']].values

# создадим Scaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_addresses)

# У нас два вектора. Сократим до одного, оставив наиболее значимую информацию.
pca = PCA(n_components=1)
pca.fit(scaled_data)
pca_data = pca.transform(scaled_data)
data['pca_address'] = pca_data
data['pca_address'] = data['pca_address'] + 5
data['pca_address'] = data['pca_address'].apply(lambda x: np.log(x) + 1)

# удалим ненужные столбцы
data.drop(['home_address','work_address'], axis=1, inplace=True)

In [238]:
label_encoder = LabelEncoder()
for col in bin_cols:
    data[col] = label_encoder.fit_transform(data[col])
data['education'] = pd.Series(label_encoder.fit_transform(data['education']))

In [239]:
# новый список колонок
num_cols = ['age','decline_app_cnt','score_bki','income','bki_request_cnt','days',
            'mean_income_age','region_rating','max_income_age', 'normalized_income',
            'mean_requests_age', 'mean_requests_income', 'mean_income_region'] 
cat_cols = ['education','pca_address','sna','first_time'] 
bin_cols = ['sex','car','car_type','good_work','foreign_passport'] 

Итак, что у нас по значимости признаков

In [240]:
imp_num = pd.Series(f_classif(data[num_cols], data['default'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

а по бинарным и категориальным

In [241]:
imp_cat = pd.Series(mutual_info_classif(data[bin_cols + cat_cols], data['default'],
                                     discrete_features =True), index = bin_cols + cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

In [242]:
data[num_cols] = pd.DataFrame(StandardScaler().fit_transform(data[num_cols]), columns = data[num_cols].columns)

In [243]:
data = pd.get_dummies(data, prefix=cat_cols, columns=cat_cols)

Теперь можно и модель построить, подобрав параметы с помощью RandomizedSearchCV

In [244]:
train_data = data.query('sample == 1').drop(['sample','client_id'], axis=1)
test_data = data.query('sample == 0').drop(['sample','client_id'], axis=1)



X = train_data.drop(['default'], axis=1).values
Y = train_data['default'].values 



X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=300)

lr = LogisticRegression(penalty = 'l2', C = 10000.0, max_iter = 1000, solver='sag')

lr.fit(X_train, Y_train)
y_pred = lr.predict(X_test)

In [245]:
probs = lr.predict_proba(X_test)
probs = probs[:,1]


fpr, tpr, threshold = roc_curve(Y_test, probs)
roc_auc = roc_auc_score(Y_test, probs)

plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label = 'Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc = 'lower right')
plt.show()

In [246]:
print('accuracy_score:',accuracy_score(Y_test,y_pred))
print('precision_score:',precision_score(Y_test,y_pred))
print('recall_score:',recall_score(Y_test,y_pred))
print('f1_score:',f1_score(Y_test,y_pred))

In [247]:
X_test = test_data.drop(['default'], axis=1)
cm = confusion_matrix(Y_test, y_pred)
cmd = ConfusionMatrixDisplay(cm, display_labels=['non_default','default'])
cmd.plot()
cmd.ax_.set(xlabel='Predicted', ylabel='True')

In [248]:
# from sklearn.model_selection import GridSearchCV

# # Добавим типы регуляризации
# penalty = ['l1', 'l2']

# # Зададим ограничения для параметра регуляризации
# C = np.logspace(0, 4, 10)

# # Создадим гиперпараметры
# hyperparameters = dict(C=C, penalty=penalty)

# model = LogisticRegression(max_iter = 1000)
# model.fit(X_train, Y_train)

# # Создаем сетку поиска с использованием 5-кратной перекрестной проверки
# clf = GridSearchCV(model, hyperparameters, cv=5, verbose=0)

# best_model = clf.fit(X_train, Y_train)

# # View best hyperparameters
# print('Лучшее Penalty:', best_model.best_estimator_.get_params()['penalty'])
# print('Лучшее C:', best_model.best_estimator_.get_params()['C'])

In [249]:
# from sklearn.model_selection import RandomizedSearchCV

# # Добавим типы регуляризации
# penalty = ['l1', 'l2']

# # Зададим ограничения для параметра регуляризации
# C = np.logspace(0, 4, 10)
# solver = ['lbfgs', 'liblinear', 'sag', 'saga']

# # Создадим гиперпараметры
# hyperparameters = dict(C=C, penalty=penalty, solver = solver)

# model = LogisticRegression(max_iter = 1000)
# model.fit(X_train, Y_train)

# # Создаем сетку поиска с использованием 5-кратной перекрестной проверки
# clf = RandomizedSearchCV(model, hyperparameters, cv=5, verbose=0)

# best_model = clf.fit(X_train, Y_train)

# # View best hyperparameters
# print('Лучшее Penalty:', best_model.best_estimator_.get_params()['penalty'])
# print('Лучшее C:', best_model.best_estimator_.get_params()['C'])
# print('Лучшее solver:', best_model.best_estimator_.get_params()['solver'])

In [250]:
# Лучшее Penalty: l2
# Лучшее C: 10000.0
# Лучшее solver: sag

In [251]:
X_test = test_data.drop(['default'], axis=1)
y_pred = lr.predict_proba(X_test)
results_df = pd.DataFrame(data={'client_id':test['client_id'], 'default':y_pred[:,1]})
results_df.to_csv('submission.csv', index=False)
results_df

### Вывод

После обработки датасета и добавления новых признаков была сгенерирована базовая модель логистической регрессии, которая показала ROC AUC > 0.74, а также accuracy 0.88, precision 0.43, recall 0.31 и f1_score 0.66.

Для подбора гиперпараметров через GridSearchCV и RandomizedSearchCV использовались LogisticRegression. 